#### Graph Neural Networks

the paper called [Graph Signal Processing for Geometric Data and
Beyond: Theory and Applications](https://arxiv.org/pdf/2008.01918) worth noting in this subject.

- Graph domains:

<img src=../images/graph_domain.png width=800>

the concept of node embedding and edge embeddings.

Adjacency Matrix($A$), Node embedding Matrix ($X$) and Edge embedding Matrix($E$):

- the properties of *Adjacency Matrix*:
    - In general, if we raise the adjacency matrix to the power of $L$, the entry at position $(m,n)$ of $A^L$ contains the number of unique walks of length $L$ from node $m$ to node $n$, by the way sometimes This is not the same as the number of unique paths since the walks include routes that visit the same node more than once. Nonetheless, $A^L$ still contains valuable information about the graph connectivity; a non-zero entry at position $(m,n)$ indicates that the distance from $m$ to $n$ must be less than or equal to $L$.

permutation of node indices in graphs -> in contrast with images where the permutation of the pixels results in changing the whole images and hence results in completely new images

The operation of exchanging node indices can be expressed mathematically by a permutation matrix, $P$. This is a matrix where exactly one entry in each row and
column take the value one, and the remaining values are zero. When position $(m,n)$ of the permutation matrix is set to one, it indicates that node $m$ will become node $n$ after the permutation. To map from one indexing to another, we use the operations:

$$
X' = XP
$$
$$
A' = P^T AP,
$$

where post-multiplying by $P$ permutes the columns and pre-multiplying by $P_T$ permutes
the rows.

#### Graph neural networks, input, output, tasks and loss-function:

A graph neural network is a model that takes the node embeddings $X$ and the adjacency
matrix $A$ as inputs and passes them through a series of $K$ layers. The node embeddings
are updated at each layer to create intermediate “hidden” representations $H_k$ before
finally computing output embeddings $H_K$

$?$ similarity with word-embed and transformers architecture:

At the start of this network, each column of the input node embeddings $X$ just contains information about the node itself. At the end, each column of the model output $H_K$ includes information about the node and its context within the graph. This is similar to
word embeddings passing through a transformer network. These represent words at the
start, but represent the word meanings in the context of the sentence at the end.

Supervised graph problems usually fall into one of three categories:

1. Graph-level tasks: The network assigns a label or estimates one or more values from
the entire graph, exploiting both the structure and node embeddings. For example, we
might want to predict the temperature at which a molecule becomes liquid (a regression
task) or whether a molecule is poisonous to human beings or not (a classification task).
For graph-level tasks, the output node embeddings are combined (e.g., by averaging),
and the resulting vector is mapped via a linear transformation or neural network to a
fixed-size vector. For regression, the mismatch between the result and the ground truth
values is computed using the least squares loss. For binary classification, the output
is passed through a sigmoid function, and the mismatch is calculated using the binary
cross-entropy loss. Here, the probability that the graph belongs to class one might be
given by:
$$
\Pr(y = 1 | X, A) = \text{sig}[\beta_K + \omega_K H_K 1/N],
$$

where the scalar $\beta_k$ and $1 \times D$ vector $\omega_k$ are learned parameters. Post-multiplying the
output embedding matrix $H_K$ by the column vector $1$ that contains ones has the effect
of summing together all the embeddings and subsequently dividing by the number of
nodes $N$ computes the average. This is known as *mean pooling*

2. Node-level tasks: The network assigns a label (classification) or one or more values
(regression) to each node of the graph, using both the graph structure and node embeddings. For example, given a graph constructed from a 3D point cloud, the goal might be to classify the nodes according to whether they belong
to the wings or fuselage. Loss functions are defined in the same way as for graph-level
tasks, except that now this is done independently at each node $n$:
$$
\operatorname{Pr}(y^{(n)} = 1 | \mathbf{X}, \mathbf{A}) = \operatorname{sig} \left[ \beta \mathbf{k} + \mathbf{w} \mathbf{h}_K^{(n)} \right]. \\
$$

3. Edge prediction tasks: The network predicts whether or not there should be an edge
between nodes n and m. For example, in the social network setting, the network might
predict whether two people know and like each other and suggest that they connect if
that is the case. This is a binary classification task where the two node embeddings must
be mapped to a single number representing the probability that the edge is present. One
possibility is to take the dot product of the node embeddings and pass the result through
a sigmoid function to create the probability:

$$\operatorname{Pr}(y^{(m)} = 1 | \mathbf{X}, \mathbf{A}) = \operatorname{sig} \left[ \mathbf{h}^{(m) \top} \mathbf{h}^{(n)} \right].$$


### Graph convolutional network:

#### spatial-based convolutional graph neural networks, or GCNs for short:
These models are convolutional
in that they update each node by aggregating information from nearby nodes. As such,
they induce a *relational inductive bias* (i.e., a bias toward prioritizing information from
neighbors). They are **spatial-based** because they use the original graph structure. This
contrasts with **spectral-based** methods, which apply convolutions in the Fourier domain.

Each layer of the GCN is a function $F[\bullet]$ with parameters $\phi$ that takes the node
embeddings and adjacency matrix and outputs new node embeddings. The network can
hence be written as:
$$
\mathbf{H}_1 = \mathbf{F}[\mathbf{X}, \mathbf{A}, \phi_0] \\

\mathbf{H}_2 = \mathbf{F}[\mathbf{H}_1, \mathbf{A}, \phi_1] \\

\mathbf{H}_3 = \mathbf{F}[\mathbf{H}_2, \mathbf{A}, \phi_2] \\

\dots \\

\mathbf{H}_K = \mathbf{F}[\mathbf{H}_{K-1}, \mathbf{A}, \phi_{K-1}]
$$

where X is the input, A is the adjacency matrix, Hk contains the modified node embeddings at the kth layer, and ϕk denotes the parameters that map from layer k to
layer k+1.

Equivariance and invariance to permutations:


$$ \mathbf{H}_{k+1}\mathbf{P} = \mathbf{F}[\mathbf{H}_k\mathbf{P}, \mathbf{P}^T\mathbf{AP}, \phi_k]. $$

$$ y = \text{sig}\left[\beta_k + \omega_k \mathbf{H}_k 1 / N\right] = \text{sig}\left[\beta_k + \omega_k \mathbf{H}_k \mathbf{P} 1 / N\right], $$


- simple graph CNN Layer:

<img src=../images/simple_graph_CNN.png width=750>

Example of simple GCN Layer:

$$ \text{agg}[n, k] = \sum_{m \in \text{NE}[n]} \mathbf{h}_k^{(m)}, $$

$$ \mathbf{h}_{k+1}^{(n)} = \text{a} \left[ \beta_k + \Omega_k \cdot \mathbf{h}_k^{(n)} + \Omega_k \cdot \text{agg}[n, k] \right]. $$

We can write this more succinctly by noting that post-multiplication of a matrix
by a vector returns a weighted sum of its columns. The $n^th$ column of the adjacency
matrix $A$ contains ones at the positions of the neighbors. Hence, if we collect the node embeddings into the $D \times N$ matrix $H_k$ and post-multiply by the adjacency matrix $A$, the nth column of the result is $agg[n,k]$. The update for the nodes is now:

$$
\begin{align}
\mathbf{H}_{k+1} &= \text{a} \left[ \beta_k \mathbf{1}^T + \Omega_k \mathbf{H}_k + \Omega_k \mathbf{H}_k \mathbf{A} \right] \\
&= \text{a} \left[ \beta_k \mathbf{1}^T + \Omega_k \mathbf{H}_k (\mathbf{A} + \mathbf{I}) \right],
\end{align}
$$

so, This layer satisfies the design considerations: it is equivariant to permutations of the node indices, can cope with any number of neighbors, exploits the graph structure to provide a relational inductive bias, and shares parameters throughout the graph.

##### 1. graph classification:
$$
\begin{align*}
\mathbf{H}_1 &= a\left[ \boldsymbol{\beta}_0 \mathbf{1}^\top + \boldsymbol{\Omega}_0 \mathbf{X} (\mathbf{A} + \mathbf{I}) \right] \\
\mathbf{H}_2 &= a\left[ \boldsymbol{\beta}_1 \mathbf{1}^\top + \boldsymbol{\Omega}_1 \mathbf{H}_1 (\mathbf{A} + \mathbf{I}) \right] \\
&\vdots \\
\mathbf{H}_K &= a\left[ \boldsymbol{\beta}_{K-1} \mathbf{1}^\top + \boldsymbol{\Omega}_{K-1} \mathbf{H}_{K-1} (\mathbf{A} + \mathbf{I}) \right] \\
f[\mathbf{X}, \mathbf{A}, \boldsymbol{\Phi}] &= \sigma\left( \boldsymbol{\beta}_K + \boldsymbol{\omega}_K \mathbf{H}_K \frac{\mathbf{1}}{N} \right)
\end{align*}
$$

where:

$\mathbf{H}_{k-1} (\mathbf{A} + \mathbf{I})$ performs a message aggregation step across the graph:

$$\left[\mathbf{H}{k-1} (\mathbf{A} + \mathbf{I})\right]{:, v} = \sum_{u \in \mathcal{N}(v) \cup \{v\}} \mathbf{h}_{u}^{(k-1)}$$

where:
- $\mathbf{H}_{k-1} \in \mathbb{R}^{D \times N}$: each column $\mathbf{h}_v$ is the embedding of node $v$ at layer $k-1$
- $\mathbf{A} + \mathbf{I} \in \mathbb{R}^{N \times N}$: adjacency matrix with self-loops.

So the new embedding of node $v$ is computed by summing up (or linearly combining) the embeddings of its neighbors $\mathcal{N}(v)$, plus itself (because of $\mathbf{I}$).


to further undrestand the concept, think of this in isolation and without the Identity function($I$), the equation is something like:
$$
(\mathbf{A}){uv} =
\begin{cases}
1 & \text{if node } u \text{ is connected to } v \\
0 & \text{otherwise}
\end{cases}
\Rightarrow \left[\mathbf{H}{k-1} \mathbf{A}\right]{:, v} = \sum{u \in \mathcal{N}(v)} \mathbf{h}^{(k-1)}_u
$$

so the node $v$ is aggregating it's neighbors embeddings.

##### Inductive and Tranductive:

wikipedia: In logic, statistical inference, and supervised learning, *transduction* or transductive inference is reasoning from observed, specific cases to specific cases. In contrast, *induction* is reasoning from observed training cases to general rules, which are then applied to the test cases. The distinction is most interesting in cases where the predictions of the transductive model are not achievable by any inductive model.

**inductive**: we exploid a training
set of labeled data to learn the relation between the inputs and outputs. Then we apply this to new test data. One way to think of this is that we are learning the rule that maps inputs to outputs and then applying it elsewhere.

**transductive**: in contrast, the transductive model considers both labeled and unlabeled data at the same time. It does not produce a rule but merely a labeling for the unknown outputs. This is sometimes termed *semi-supervised* learning. It has the advantage that it can use patterns in the unlabeled data to help make its decisions. However, it has the disadvantage that the model needs to be retrained when extra unlabeled data are added.

Both problem types are commonly encountered for graphs. Sometimes,
we have many labeled graphs and learn a mapping between the graph and the labels.
For example, we might have many molecules, each labeled according to whether it is
toxic to humans. We learn the rule that maps the graph to the toxic/non-toxic label and
then apply this rule to new molecules. However, sometimes there is a single monolithic
graph. In the graph of scientific paper citations, we might have labels indicating the field
(physics, biology, etc.) for some nodes and wish to label the remaining nodes. Here, the
training and test data are irrevocably connected. Graph-level tasks only occur in the inductive setting where there are training and test
graphs. However, node-level tasks and edge prediction tasks can occur in either setting.
In the transductive case, the loss function minimizes the mismatch between the model
output and the ground truth where this is known. New predictions are computed by
running the forward pass and retrieving the results where the ground truth is unknown.

##### 2. Node classification

in this category, Training this network raises two problems. First, it is logistically diﬀicult to train a
graph neural network of this size. Consider that we must store the node embeddings at
every network layer in the forward pass. This will involve both storing and processing
a structure several times the size of the entire graph, and this may not be practical.
Second, we have only a single graph, so it’s not obvious how to perform stochastic
gradient descent. How can we form a batch if there is only a single object?

**choosing a batch**:

One way to form a batch is to choose a random subset of labeled nodes at each training
step. Each node depends on its neighbors in the previous layer. These, in turn, depend on their neighbors in the layer before, so (similarly to convolutional networks) each node has a receptive field. The receptive field size is termed the *k-hop neighborhood*.
We can hence perform a gradient descent step using the graph that forms the union of
the k-hop neighborhoods of the batch nodes; the remaining inputs do not contribute. Unfortunately, if there are many layers and the graph is densely connected, every
input node may be in the receptive field of every output, and this may not reduce the
graph size at all. This is known as the *graph expansion problem*. Two approaches that
tackle this problem are neighborhood sampling and graph partitioning.

##### Some notes on the [GCN from Microsoft](https://youtu.be/zCEYiCxrL_0?si=7cE5t0IahAtWSCnF)

- the process of updating nodes:

<img src=../images/microsoft_GCN1.png width=650>